Coursera capstone

In [1]:
zomatao_user_key = 'b348f057e5edb73209c2caa7f6dcb638'
kol_geojson = 'https://raw.githubusercontent.com/datameet/Municipal_Spatial_Data/master/Kolkata/kolkata.geojson'

In [2]:
# Zomato web acesss
class Zomato:
    def __init__(self, user_key, content_type = 'application/json'):
        self._key = user_key
        self.request_count = 0
        self.content_type = content_type
        self._header = {
                    'Accept': content_type,
                    'user-key': user_key,
                            }
        self._categories_id = {}

    def get_response(self,api,params=''):
        import requests
        headers = self._header
        status = {  200 :'Sucessfully Connected to Zomato..',
                    403 :'Connection Error: Invalid User Keys',
                    404 : 'No Connection to Server..'
                }
        response = requests.get(api, headers=headers,params=params)
        print(status[response.status_code])
        if response.status_code == 200:
            self.request_count += 1
            return response.json()
    
    def categories(self,method='get'):
        api = 'https://developers.zomato.com/api/v2.1/categories'
        response = self.get_response(api)
        result_dict = {categories['categories']['name']: categories['categories']['id'] for categories in response['categories']}
        if method == 'set':
            self._categories_id = result_dict 
        return result_dict
    
    def cities(self,city_name='',id='',lat='',lon=''):
        params = (
            ('q', city_name),
            ('lat', lat),
            ('lon', lon),
            ('city_ids', id))

        api = 'https://developers.zomato.com/api/v2.1/cities'
        response = self.get_response(api,params=params)  
        return response

    def cuisines(self,city_id='',lat='',lon=''):
        params = (
            ('city_id', city_id),
            ('lat', lat),
            ('lon', lon))
        api = 'https://developers.zomato.com/api/v2.1/cuisines'
        response = self.get_response(api,params=params)  
        return response
    
    def geocode(self,lat='',lon=''):
        api = 'https://developers.zomato.com/api/v2.1/geocode'
        params = (
            ('lat', lat),
            ('lon', lon))
        response = self.get_response(api,params=params)  
        return response

    def location(self,city='',lat='',lon='',max_count=100):
        api = 'https://developers.zomato.com/api/v2.1/locations'
        params = (
        ('query', city),
        ('lat', lat),
        ('lon', lon),
        ('count', str(max_count)))
        response = self.get_response(api,params=params)  
        return response
    
    def location_details(self,city='',lat='',lon=''):
        r = self.location(city,lat,lon,1)
        entity_id = r['location_suggestions'][0]['entity_id']
        entity_type = r['location_suggestions'][0]['entity_type']

        params = (
            ('entity_id', entity_id),
            ('entity_type', entity_type))
        api = 'https://developers.zomato.com/api/v2.1/location_details'

        response = self.get_response(api,params=params)  
        return response
    
    def restaurant(self,res_id=''):
        api = 'https://developers.zomato.com/api/v2.1/restaurant'
        params = (
                ('res_id', '16765362'),)
        response = self.get_response(api,params=params) 
        return response
    
    def reviews(self,res_id='',start='1',count='10'):
        api = 'https://developers.zomato.com/api/v2.1/reviews'
        params = (
            ('res_id', res_id),
            ('start', start),
            ('count', count),) 
        response = self.get_response(api,params=params) 
        return response


In [3]:
zomato = Zomato(zomatao_user_key)

In [4]:
# kolkata pincode corrdinates
from bs4 import BeautifulSoup #Web Scrapping
import pandas as pd #library for dataframe
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import numpy as np
from tabulate import tabulate
import json #library for json file
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium # map rendering library

In [5]:
with open("pincode.html", "r") as f:
    
    contents = f.read()

    soup = BeautifulSoup(contents, 'lxml')

In [37]:
pincodes = soup.find_all('div',class_='jbzYp')
pincodes = [pc.string for pc in pincodes]

cordinates = soup.find_all('div',class_='title')
cordinates = [(c.string.split(", ")[0][:7],c.string.split(", ")[1][:7]) for c in cordinates]

kolkata_pincodes = pd.DataFrame({'pincode':pincodes,'cordinates':cordinates}).sort_values(by='pincode')
kolkata_pincodes

,pincode,cordinates
0,700001,"(22.5736, 88.3483)"
4,700002,"(22.6189, 88.3732)"
41,700005,"(22.5981, 88.3581)"
9,700006,"(22.5891, 88.3688)"
7,700007,"(22.5826, 88.3617)"
50,700008,"(22.4812, 88.3135)"
26,700009,"(22.5767, 88.3735)"
42,700012,"(22.5684, 88.3616)"
14,700014,"(22.5596, 88.3705)"
15,700016,"(22.5535, 88.3566)"
